In [1]:
import os

from paper.utils import setup_logging

from paper.utils import load_rat_data


DIR = "/mount/hdd1/acute_mapping/proc/physio"
SUBJECTS = range(1, 9)


In [2]:
unique_subdirs = []

for subject in SUBJECTS:
    curr_dir = os.path.join(DIR, f"amap{subject:002}")
    unique_subdirs += os.listdir(curr_dir)

unique_subdirs = list(set(unique_subdirs))
len(unique_subdirs)


75

## J device

    1. J_RCML_000   -- J_RCML

    2. J_SHAP_000   -- J_SHAP
    3. J_SHAP_001   -- J_SHAP



In [3]:
j_dirs = [d for d in unique_subdirs if "J" in d]
len(j_dirs)


32

In [4]:
j_rcml_dirs = [d for d in j_dirs if "RCML" in d]
print(len(j_rcml_dirs))

j_rcml_dirs


16


['2023-04-04_J_RCML_002_J1_lateral',
 '2023-04-04_J_RCML_005_J7_lateral',
 '2023-03-29_J_RCML_000',
 '2023-03-27_J_RCML_000',
 '2023-04-03_J_RCML_002_mid',
 '2023-03-17_J_RCML_000',
 '2023-03-13_J_RCML_000',
 '2023-04-03_J_RCML_003_norm',
 '2023-04-04_J_RCML_000',
 '2023-03-21_J_RCML_000',
 '2023-04-03_J_RCML_000',
 '2023-04-04_J_RCML_004_J7_midline',
 '2023-04-04_J_RCML_001_J1_midline',
 '2023-03-24_J_RCML_000',
 '2023-04-04_J_RCML_003_J7_reduced',
 '2023-04-03_J_RCML_001_lat']

In [5]:
j_shap_dirs = [d for d in j_dirs if "SHAP" in d]
print(len(j_shap_dirs))

j_shap_dirs


16


['2023-03-29_J_SHAP_000',
 '2023-04-04_J_SHAP_001',
 '2023-03-21_J_SHAP_000',
 '2023-03-24_J_SHAP_000',
 '2023-03-17_J_SHAP_000',
 '2023-03-27_J_SHAP_001',
 '2023-03-13_J_SHAP_001',
 '2023-03-17_J_SHAP_001',
 '2023-04-04_J_SHAP_000',
 '2023-03-13_J_SHAP_000',
 '2023-04-03_J_SHAP_000',
 '2023-03-24_J_SHAP_001',
 '2023-03-21_J_SHAP_001',
 '2023-03-29_J_SHAP_001',
 '2023-04-03_J_SHAP_001',
 '2023-03-27_J_SHAP_000']

### J_RCML data

In [6]:
subdir_pattern = ["*J_RCML_000*"]
subjects = range(1, 9)

df, mat, time, auc_window, muscles_map = load_rat_data(
    dir=DIR, subdir_pattern=subdir_pattern, subjects=SUBJECTS
)


100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  2.74it/s]


### J_SHAP data

In [7]:
subdir_pattern = ["*J_SHAP*"]

df, mat, time, auc_window, muscles_map = load_rat_data(
    dir=DIR, subdir_pattern=subdir_pattern, subjects=SUBJECTS
)

df.pulse_amplitude = df.apply(
        lambda x: (x.pulse_amplitude * 4)
        if x.compound_charge_params == "80-0-20-400"
        else x.pulse_amplitude,
        axis=1
    )


100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.24it/s]


In [12]:
import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal


src = "/home/vishu/data/hbmep-processed/rat/J_SHAP/data.csv"
comp_df = pd.read_csv(src)
comp_df = comp_df.rename(columns=muscles_map).copy()

src = "/home/vishu/data/hbmep-processed/rat/J_SHAP/mat.npy"
comp_mat = np.load(src)


In [13]:
assert_frame_equal(df, comp_df)


In [14]:
(mat == comp_mat).all()


True